# COGS 108 - EDA Checkpoint

# Names

- Ant Man
- Hulk
- Iron Man
- Thor
- Wasp

<a id='research_question'></a>
# Research Question

*Fill in your research question here*

# Setup

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import patsy
import statsmodels.api as sm
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.style as style

# Data Cleaning

The raw data requires a hefty amount of wrangling due to the fact that the raw data is incredibly untidy (as seen in the first cell below). For starters, the Metropolitan Statistical Area (MSA) is always listed on one line with the next few lines being NaNs since it was made for humans to see and understand that the NaNs were meant to still correspond to the previously listed MSA. To further throw a wrench in things many of the lines following each MSA aren't even data, such as the line after each one that says which counties are included in the MSA. Next, the columns are basically all unnamed because the actual names for them are all in row 2 (and overall rows 0-2 are basically just gibberish). Furthermore, they list the murders for each city in the MSA, but only list the rate per 100k inhabitants for the whole MSA. Finally, there were many different crimes listed but the one we are focusing on are murder rates. Thus, to wrangle all the data for each year I had to drop many of the columns, and rows so that it is only the murder rates per city and per 100k inhabitants in each MSA, as well as the population of each city and MSA. To finish up the wrangling I split up each year into two datasets: one for each city's murders and one for each MSA's murder rate per 100k inhabitants.

In [2]:
untidyShowcase = pd.read_csv('https://raw.githubusercontent.com/azhong19/COGS108_Repo/main/2015.xls%20-%2015tbl06.csv')
untidyShowcase.iloc[0:15]

,Table 6,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Crime in the United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"by Metropolitan Statistical Area, 2015",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Metropolitan Statistical Area,Counties/principal cities,Population,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape1,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft
3,"Abilene, TX M.S.A.",NaN,"169,713",,NaN,NaN,NaN,NaN,,NaN,NaN,NaN
4,NaN,"Includes Callahan, Jones, and Taylor Counties",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,City of Abilene,"121,764",613,7,84,128,394,"5,354","1,168","3,798",388
6,NaN,Total area actually reporting,100.0%,700,9,95,133,463,"6,125","1,446","4,232",447
7,NaN,"Rate per 100,000 inhabitants",NaN,412.5,5.3,56.0,78.4,272.8,"3,609.0",852.0,"2,493.6",263.4
8,"Akron, OH M.S.A.",NaN,"704,320",,NaN,NaN,NaN,NaN,,NaN,NaN,NaN
9,NaN,Includes Portage and Summit Counties,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#initial csv reading and wrangling (2015)
df2015 = pd.read_csv('https://raw.githubusercontent.com/azhong19/COGS108_Repo/main/2015.xls%20-%2015tbl06.csv')
df2015 = df2015.drop(df2015.columns[[3, 5, 6, 7, 8, 9, 10, 11]], axis = 1)
df2015.columns = ['Area', 'City', 'Population', 'Murders']
df2015 = df2015[(df2015['City'].str.contains("City of")) | (df2015['City'].str.contains("Rate per")) | (df2015['Area'].str.contains("M.S.A")) | (df2015['Area'].str.contains("M.D"))]
df2015 = df2015.reset_index(drop=True)

#creating a joined dataset for Metropolitan Statistical Area and Murders per 100k inhabitants
area2015 = df2015[['Area', 'Population']]
area2015 = area2015.dropna().reset_index(drop=True)
rate2015 = df2015[['City', 'Murders']]
rate2015 = rate2015[(rate2015['City'].str.contains("Rate per")) | (rate2015['City'].str.contains("Rate per"))] #for some reason I have to have the or statement here or it doesn't work, I know it looks incredibly jank but I couldn't think of a better way to do it
rate2015 = rate2015.reset_index(drop=True)
joined2015 = area2015.join(rate2015, lsuffix='Area', rsuffix='City')
joined2015 = joined2015.drop(joined2015.columns[[2]], axis = 1)
joined2015.columns = ['Area', 'Population', 'Rate (per 100k inhabitants)']

#creating a dataset for murders per city
city2015 = df2015[['City', 'Population', 'Murders']]
city2015 = city2015[(city2015['City'].str.contains("City of")) | (city2015['City'].str.contains("City of"))] #again have to have the or statement here, this is gonna make me lose what little sanity I had left
city2015 = city2015.reset_index(drop=True)
city2015

,City,Population,Murders
0,City of Abilene,"121,764",7
1,City of Akron,"197,587",28
2,City of Albany,"75,357",10
3,City of Albany,"52,394",2
4,City of Albuquerque,"559,721",43
...,...,...,...
556,City of Yakima,"93,798",7
557,City of York,"43,864",10
558,City of Hanover,"15,494",0
559,City of Yuba City,"65,974",0


In [4]:
#initial csv reading and wrangling (2016)
df2016 = pd.read_csv('https://raw.githubusercontent.com/azhong19/COGS108_Repo/main/2016.xls%20-%2016tbl04.csv')
df2016 = df2016.drop(df2016.columns[[3, 5, 6, 7, 8, 9, 10, 11]], axis = 1)
df2016.columns = ['Area', 'City', 'Population', 'Murders']
df2016 = df2016[(df2016['City'].str.contains("City of")) | (df2016['City'].str.contains("Rate per")) | (df2016['Area'].str.contains("M.S.A")) | (df2016['Area'].str.contains("M.D"))]
df2016 = df2016.reset_index(drop=True)

#creating a joined dataset for Metropolitan Statistical Area and Murders per 100k inhabitants
area2016 = df2016[['Area', 'Population']]
area2016 = area2016.dropna().reset_index(drop=True)
rate2016 = df2016[['City', 'Murders']]
rate2016 = rate2016[(rate2016['City'].str.contains("Rate per")) | (rate2016['City'].str.contains("Rate per"))] #for some reason I have to have the or statement here or it doesn't work, I know it looks incredibly jank but I couldn't think of a better way to do it
rate2016 = rate2016.reset_index(drop=True)
joined2016 = area2016.join(rate2016, lsuffix='Area', rsuffix='City')
joined2016 = joined2016.drop(joined2016.columns[[2]], axis = 1)
joined2016.columns = ['Area', 'Population', 'Rate (per 100k inhabitants)']

#creating a dataset for murders per city
city2016 = df2016[['City', 'Population', 'Murders']]
city2016 = city2016[(city2016['City'].str.contains("City of")) | (city2016['City'].str.contains("City of"))] #again have to have the or statement here, this is gonna make me lose what little sanity I had left
city2016 = city2016.reset_index(drop=True)
city2016

,City,Population,Murders
0,City of Abilene,"122,523",8
1,City of Akron,"197,257",34
2,City of Albany,"74,330",16
3,City of Albany,"52,544",0
4,City of Albany,"98,617",2
...,...,...,...
568,"City of Worcester, MA","184,595",8
569,City of Yakima,"94,111",13
570,City of York,"44,017",8
571,City of Hanover,"15,537",0


In [5]:
#initial csv reading and wrangling (2017)
df2017 = pd.read_csv('https://raw.githubusercontent.com/azhong19/COGS108_Repo/main/2017%20crime.csv')
df2017 = df2017.drop(df2017.columns[[3, 5, 6, 7, 8, 9, 10, 11]], axis = 1)
df2017.columns = ['Area', 'City', 'Population', 'Murders']
df2017 = df2017[(df2017['City'].str.contains("City of")) | (df2017['City'].str.contains("Rate per")) | (df2017['Area'].str.contains("M.S.A")) | (df2017['Area'].str.contains("M.D"))]
df2017 = df2017.reset_index(drop=True)

#creating a joined dataset for Metropolitan Statistical Area and Murders per 100k inhabitants
area2017 = df2017[['Area', 'Population']]
area2017 = area2017.dropna().reset_index(drop=True)
rate2017 = df2017[['City', 'Murders']]
rate2017 = rate2017[(rate2017['City'].str.contains("Rate per")) | (rate2017['City'].str.contains("Rate per"))] #for some reason I have to have the or statement here or it doesn't work, I know it looks incredibly jank but I couldn't think of a better way to do it
rate2017 = rate2017.reset_index(drop=True)
joined2017 = area2017.join(rate2017, lsuffix='Area', rsuffix='City')
joined2017 = joined2017.drop(joined2017.columns[[2]], axis = 1)
joined2017.columns = ['Area', 'Population', 'Rate (per 100k inhabitants)']

#creating a dataset for murders per city
city2017 = df2017[['City', 'Population', 'Murders']]
city2017 = city2017[(city2017['City'].str.contains("City of")) | (city2017['City'].str.contains("City of"))] #again have to have the or statement here, this is gonna make me lose what little sanity I had left
city2017 = city2017.reset_index(drop=True)
city2017

,City,Population,Murders
0,"City of Abilene2, 3","122,981",4
1,City of Akron,"197,412",42
2,City of Albany,"73,209",22
3,City of Albany2,"53,701",0
4,City of Albany,"98,174",8
...,...,...,...
591,City of Yakima,"94,375",13
592,City of York,"43,857",14
593,City of Hanover,"15,606",0
594,City of Yuba City,"67,042",0


In [6]:
#initial csv reading and wrangling (2018)
df2018 = pd.read_csv('https://raw.githubusercontent.com/azhong19/COGS108_Repo/main/2018.xls%20-%2018tbl06.csv')
df2018 = df2018.drop(df2018.columns[[3, 5, 6, 7, 8, 9, 10, 11]], axis = 1)
df2018.columns = ['Area', 'City', 'Population', 'Murders']
df2018 = df2018[(df2018['City'].str.contains("City of")) | (df2018['City'].str.contains("Rate per")) | (df2018['Area'].str.contains("M.S.A")) | (df2018['Area'].str.contains("M.D"))]
df2018 = df2018.reset_index(drop=True)

#creating a joined dataset for Metropolitan Statistical Area and Murders per 100k inhabitants
area2018 = df2018[['Area', 'Population']]
area2018 = area2018.dropna().reset_index(drop=True)
rate2018 = df2018[['City', 'Murders']]
rate2018 = rate2018[(rate2018['City'].str.contains("Rate per")) | (rate2018['City'].str.contains("Rate per"))] #for some reason I have to have the or statement here or it doesn't work, I know it looks incredibly jank but I couldn't think of a better way to do it
rate2018 = rate2018.reset_index(drop=True)
joined2018 = area2018.join(rate2018, lsuffix='Area', rsuffix='City')
joined2018 = joined2018.drop(joined2018.columns[[2]], axis = 1)
joined2018.columns = ['Area', 'Population', 'Rate (per 100k inhabitants)']

#creating a dataset for murders per city
city2018 = df2018[['City', 'Population', 'Murders']]
city2018 = city2018[(city2018['City'].str.contains("City of")) | (city2018['City'].str.contains("City of"))] #again have to have the or statement here, this is gonna make me lose what little sanity I had left
city2018 = city2018.reset_index(drop=True)
city2018

,City,Population,Murders
0,City of Abilene,"122,480",8
1,City of Akron,"197,690",38
2,City of Albany,"72,594",15
3,City of Albany,"98,322",12
4,City of Schenectady,"65,550",1
...,...,...,...
569,City of Yakima,"93,959",16
570,City of York,"44,170",20
571,City of Hanover,"15,653",1
572,City of Yuba City,"67,035",1


In [7]:
#initial csv reading and wrangling (2019)
df2019 = pd.read_csv('https://raw.githubusercontent.com/azhong19/COGS108_Repo/main/2019.xls%20-%2019tbl06.csv')
df2019 = df2019.drop(df2019.columns[[3, 5, 6, 7, 8, 9, 10, 11]], axis = 1)
df2019.columns = ['Area', 'City', 'Population', 'Murders']
df2019 = df2019[(df2019['City'].str.contains("City of")) | (df2019['City'].str.contains("Rate per")) | (df2019['Area'].str.contains("M.S.A")) | (df2019['Area'].str.contains("M.D"))]
df2019 = df2019.reset_index(drop=True)

#creating a joined dataset for Metropolitan Statistical Area and Murders per 100k inhabitants
area2019 = df2019[['Area', 'Population']]
area2019 = area2019.dropna().reset_index(drop=True)
rate2019 = df2019[['City', 'Murders']]
rate2019 = rate2019[(rate2019['City'].str.contains("Rate per")) | (rate2019['City'].str.contains("Rate per"))] #for some reason I have to have the or statement here or it doesn't work, I know it looks incredibly jank but I couldn't think of a better way to do it
rate2019 = rate2019.reset_index(drop=True)
joined2019 = area2019.join(rate2018, lsuffix='Area', rsuffix='City')
joined2019 = joined2019.drop(joined2019.columns[[2]], axis = 1)
joined2019.columns = ['Area', 'Population', 'Rate (per 100k inhabitants)']

#creating a dataset for murders per city
city2019 = df2019[['City', 'Population', 'Murders']]
city2019 = city2019[(city2019['City'].str.contains("City of")) | (city2019['City'].str.contains("City of"))] #again have to have the or statement here, this is gonna make me lose what little sanity I had left
city2019 = city2019.reset_index(drop=True)
city2019

,City,Population,Murders
0,City of Abilene,"123,665",6
1,City of Akron,"197,882",27
2,City of Albany3,"74,989",12
3,City of Albany,"54,993",1
4,City of Lebanon,"17,304",0
...,...,...,...
490,"City of Winchester, VA","28,201",1
491,"City of Worcester, MA","184,945",13
492,City of Yakima,"94,168",9
493,City of Yuba City,"67,164",3


Now that the data is all wrangled we can finally clean the data. The main things that need to be done are removing the random numbers in the city/MSA names as well as changing the population be an int instead of an object.

In [8]:
#removes all numbers
def standardize_city_MSA_names(string):
    
    string = string.strip()
    string = string.replace("1", "")
    string = string.replace("2", "")
    string = string.replace("3", "")
    string = string.replace("4", "")
    string = string.replace("5", "")
    string = string.replace("6", "")
    string = string.replace("7", "")
    string = string.replace("8", "")
    string = string.replace("9", "")
    string = string.replace("0", "")
    string = string.strip()
    
    return string

joined2015['Area'] = joined2015['Area'].apply(standardize_city_MSA_names)
city2015['City'] = city2015['City'].apply(standardize_city_MSA_names)

joined2016['Area'] = joined2016['Area'].apply(standardize_city_MSA_names)
city2016['City'] = city2016['City'].apply(standardize_city_MSA_names)

joined2017['Area'] = joined2017['Area'].apply(standardize_city_MSA_names)
city2017['City'] = city2017['City'].apply(standardize_city_MSA_names)

joined2018['Area'] = joined2018['Area'].apply(standardize_city_MSA_names)
city2018['City'] = city2018['City'].apply(standardize_city_MSA_names)

joined2019['Area'] = joined2019['Area'].apply(standardize_city_MSA_names)
city2019['City'] = city2019['City'].apply(standardize_city_MSA_names)
city2019

,City,Population,Murders
0,City of Abilene,"123,665",6
1,City of Akron,"197,882",27
2,City of Albany,"74,989",12
3,City of Albany,"54,993",1
4,City of Lebanon,"17,304",0
...,...,...,...
490,"City of Winchester, VA","28,201",1
491,"City of Worcester, MA","184,945",13
492,City of Yakima,"94,168",9
493,City of Yuba City,"67,164",3


In [9]:
#converts population into an int
def standardize_numbers(string):
    
    string = string.strip()
    string = string.replace(",", "")
    string = string.strip()
    output = int(string)
    
    return output

city2015['Population'] = city2015['Population'].apply(standardize_numbers)
city2016['Population'] = city2016['Population'].apply(standardize_numbers)
city2017['Population'] = city2017['Population'].apply(standardize_numbers)
city2018['Population'] = city2018['Population'].apply(standardize_numbers)
city2019['Population'] = city2019['Population'].apply(standardize_numbers)
city2019

,City,Population,Murders
0,City of Abilene,123665,6
1,City of Akron,197882,27
2,City of Albany,74989,12
3,City of Albany,54993,1
4,City of Lebanon,17304,0
...,...,...,...
490,"City of Winchester, VA",28201,1
491,"City of Worcester, MA",184945,13
492,City of Yakima,94168,9
493,City of Yuba City,67164,3


# Data Analysis & Results (EDA)

Carry out EDA on your dataset(s); Describe in this section

In [10]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION